<a href="https://colab.research.google.com/github/armandossrecife/my_validation3/blob/main/my_analysis_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup do Ambiente de Análise

In [ ]:
!rm -rf *.log
!rm -rf *.txt
!rm -rf *.xlsx
!rm -rf my_issues
!rm -rf cassandra

In [ ]:
print('Install Pydriller.')
!pip install pydriller > install_pydriller.log
print('Install gitpython.')
!pip3 install gitpython > install_gitpython.log
print('Install Jira Python lib.')
!pip install jira > install_jira_python.log
print('Install SQLite in Linux')
!sudo apt install -y sqlite3 > install_sqlite.log
print('All depenpencies installed!')
!cat *.log > install.log
print('Details in install.log')

Install Pydriller.
Install gitpython.
Install Jira Python lib.
Install SQLite in Linux


All depenpencies installed!
Details in install.log


## Importa bibliotecas necessárias

In [ ]:
import os
from datetime import datetime, timedelta
import tqdm
from pydriller import Repository
from jira import JIRA
import pandas as pd
import re
import scipy.stats as stats
import random
import shutil
import time

## Variáveis Globais

In [ ]:
my_repository = 'cassandra'
url_to_repository = 'https://github.com/apache/cassandra.git'
os.environ['MY_REPOSITORY'] = url_to_repository

JIRA_SERVER = 'https://issues.apache.org/jira'
ISSUE_TRACKER_PROJECT = 'CASSANDRA'

# Credentials
os.environ['USERNAME'] = '?'
os.environ['PASSWORD'] = '?'
username = os.environ.get('USERNAME')
password = os.environ.get('PASSWORD')

lista_arquivos_criticos = ['StorageService.java', 'ColumnFamilyStore.java','DatabaseDescriptor.java','CompactionManager.java','StorageProxy.java','SSTableReader.java','Config.java','CassandraDaemon.java','SelectStatement.java','SinglePartitionReadCommand.java','NodeProbe.java','MessagingService.java']

## Clona o Repositório

In [ ]:
print(f'Clona o repositório : {url_to_repository}')
!git clone $MY_REPOSITORY

Clona o repositório : https://github.com/apache/cassandra.git
Cloning into 'cassandra'...
remote: Enumerating objects: 405595, done.
remote: Counting objects: 100% (1158/1158), done.
remote: Compressing objects: 100% (765/765), done.
remote: Total 405595 (delta 415), reused 778 (delta 246), pack-reused 404437
Receiving objects: 100% (405595/405595), 406.65 MiB | 20.29 MiB/s, done.
Resolving deltas: 100% (236522/236522), done.
Updating files: 100% (6212/6212), done.


## Funções de apoio

In [ ]:
def get_all_commits_by_range(initial_date, final_date, repository_name):
  """Extracts information from a date range of commits
  Args:
      initial_date: inicial date of commits
      final_date: final date of commits
  Returns:
        A dictionary of commits, where the keys are the commit hashes and the
        values are tuples containing the commit message, commit's date, commit's line, commit's file, a list of files modified in the commit, a list of all diffs from modified files
  """
  dict_commit_modified_files = {}
  print('Wait...')
  my_traverser_commits = Repository(repository_name, since=initial_date, to=final_date).traverse_commits()
  total_commits = len(list(my_traverser_commits))
  try:
    for commit in tqdm.tqdm(Repository(repository_name, since=initial_date, to=final_date).traverse_commits(), total=total_commits, desc="Progress commit anlysis"):
      list_of_modified_files = []
      list_dict_of_diff_modified_files = []
      for m in commit.modified_files:
        dict_m_diff = {}
        if m is not None:
          list_of_modified_files.append(m.filename)
          dict_m_diff[m.filename] = m.diff
          list_dict_of_diff_modified_files.append(dict_m_diff)
      data_commit = str(commit.committer_date.day) + '/' + str(commit.committer_date.month) + '/' + str(commit.committer_date.year)
      element = commit.msg, data_commit, commit.lines, commit.files, list_of_modified_files, list_dict_of_diff_modified_files
      dict_commit_modified_files[commit.hash] = element
  except Exception as ex:
    print(f'Erro during travesse commits: {str(ex)}')
  return dict_commit_modified_files, total_commits

def get_commits_by_range_and_critical_files(initial_date, final_date, critical_files, repository_name):
  """Extracts information from a date range of commits, focusing on critical files.
  Args:
      initial_date: inicial date of commits
      final_date: final date of commits
      critical_files: A list of critical file paths. ex: ['StorageService.java', 'ColumnFamilyStore.java']
  Returns:
        A dictionary of commits, where the keys are the commit hashes and the
        values are tuples containing the commit message, commits's date, commit's line, commit's file, a list of critical files modified in the commit, a list_of_modified_files, a dictionary of diff [filename]:filename.diff, a list of all diffs from modified files
        commit.msg, data_commit, commit.lines, commit.files, list_of_critical_files_modified, list_of_modified_files, list_dict_of_diff_files, list_dict_of_diff_modified_files
  """

  dict_commit_modified_files = {}
  print('Wait...')
  my_traverser_commits = Repository(repository_name, since=initial_date, to=final_date).traverse_commits()
  total_commits = len(list(my_traverser_commits))
  try:
    for commit in tqdm.tqdm(Repository(repository_name, since=initial_date, to=final_date).traverse_commits(), total=total_commits, desc="Progress critical files commit anlysis"):
      list_of_critical_files_modified = []
      list_of_modified_files = []
      list_dict_of_diff_files = []
      list_dict_of_diff_modified_files = []
      for m in commit.modified_files:
        dict_diff_file = {}
        list_of_modified_files.append(m.filename)
        list_dict_of_diff_modified_files.append(m.diff)
        if m.filename in critical_files:
          list_of_critical_files_modified.append(m.filename)
          dict_diff_file[m.filename] = m.diff
          list_dict_of_diff_files.append(dict_diff_file)
      if len(list_of_critical_files_modified) > 0:
        data_commit = str(commit.committer_date.day) + '/' + str(commit.committer_date.month) + '/' + str(commit.committer_date.year)
        element = commit.msg, data_commit, commit.lines, commit.files, list_of_critical_files_modified, list_of_modified_files, list_dict_of_diff_files, list_dict_of_diff_modified_files
        dict_commit_modified_files[commit.hash] = element
  except Exception as ex:
    print(f'Erro during travesse commits: {str(ex)}')
  return dict_commit_modified_files, total_commits

In [ ]:
class JiraIssue:
  def __init__(self, key, summary, issue_type, status, priority, description, comments, created_date=None, updated_date=None, resolved_date=None):
    self.key = key
    self.summary = summary
    self.issue_type = issue_type
    self.status = status
    self.priority = priority
    self.description = description
    self.comments = comments
    self.created_date = created_date
    self.updated_date = updated_date
    self.resolved_date = resolved_date

  def get_comments(self):
    return self.comments

  def __str__(self):
    return (f'Key: {self.key}, Summary: {self.summary}, Type: {self.issue_type}, Status: {self.status}')

class JiraIssues:
  def __init__(self,project, issues):
    self.project = project
    self.issues = issues

  def add_issue(self, issue):
    self.issues.append(issue)

  def get_issues(self) -> list:
    return self.issues

  def update_issues(self, issues):
    self.issues = issues

  def __str__(self):
    str_issues = ""
    for issue in self.get_issues():
      str_issues = str_issues + str(issue)
      str_issues = str_issues + ', '
    str_issues = '[' + str_issues + ']'
    return (f'Project: {self.project}, Qdt of issues: {len(self.issues)}, Issues: {str_issues}')

# Classe de utilidades para manipular o servidor Jira
class JiraUtils:
  def __init__(self, project, jira_instance):
    self.project = project
    self.jira_jira_instance = jira_instance

  def generate_intervals_between_dates(self, date1: tuple, date2: tuple, distance=120) -> list:
    start_date = datetime(date1[0], date1[1], date1[2])
    end_date = datetime(date2[0], date2[1], date2[2])
    interval_days = distance
    # Initialize a list to store the intervals
    intervals = []
    # Initialize the current date as the start date
    current_date = start_date
    # Loop to generate intervals until the current date is less than or equal to the end date
    while current_date < end_date:
        interval = (current_date, current_date + timedelta(days=interval_days - 1))
        intervals.append(interval)
        current_date += timedelta(days=interval_days)
    return intervals

  def convert_interval_dates(self, dates: list) -> list:
    list_interval_dates = []
    for each in dates:
      date1 = each[0]
      # Convert the date to a string in the format "YYYY/MM/DD".
      str_date1 = date1.strftime("%Y/%m/%d")
      date2 = each[1]
      str_date2 = date2.strftime("%Y/%m/%d")
      elemento = str_date1, str_date2
      list_interval_dates.append(elemento)
    return list_interval_dates

  def generate_list_of_sentences(self, dates: list) -> list:
    lista_sentencas = []
    for each in dates:
      str_date1 = each[0].strftime("%Y/%m/%d")
      str_date2 = each[1].strftime("%Y/%m/%d")
      sentenca = f'project={self.project.upper()} and created>="{str_date1}" and created<="{str_date2}"'
      lista_sentencas.append(sentenca)
    return lista_sentencas

  def get_list_of_block_issues_by_dates(self,date1, date2, distance=120) -> list:
    print('Aguarde...')
    t1 = datetime.now()
    list_of_dates = self.generate_intervals_between_dates(date1,date2,distance)
    lista_sentencas = self.generate_list_of_sentences(list_of_dates)
    lista_bloco_issues_by_date = []
    total_items = len(lista_sentencas)
    i = 0
    iterable_lista_sentencas = tqdm.tqdm(lista_sentencas, total=total_items)
    for each in iterable_lista_sentencas:
      issues_by_date_temp = self.jira_jira_instance.search_issues(each,maxResults=1000)
      print(f'Range: {each}, qtd issues: {len(issues_by_date_temp)}')
      lista_bloco_issues_by_date.append(issues_by_date_temp)
      percentage = (i + 1) / total_items * 100
      iterable_lista_sentencas.set_description(f"Progress Message Analysis")
    i += 1
    t2 = datetime.now()
    print(t2)
    print(f'Tempo da consulta: {t2-t1}')
    return lista_bloco_issues_by_date

  def concatenate_block_of_issues(self,block_of_issues):
    concatenated_list = [item for sublist in block_of_issues for item in sublist]
    print(f'Total de issues recuperados: {len(concatenated_list)}')
    return concatenated_list

def analyze_jira_all_issues(project, all_issues):
    """Extracts all issues from Issue Tracker
        Args:
          project: Issue Tracker Project name, example: CASSANDRA
          all_issues: a block of issues
        Returns:
          A list of JiraIssues
    """
    # Create an instance of JiraIssues to manage all issues
    my_all_issues = JiraIssues(project, [])
    total_items = len(all_issues)

    # Iterate through the fetched issues
    for issue in tqdm.tqdm(all_issues, total=total_items, desc='Progress jira all issues analysis'):
        issue_key = issue.key
        issue_summary = issue.fields.summary
        issue_description = issue.fields.description
        issue_comments = [comment.body for comment in issue.fields.comment.comments]
        issue_created_date = issue.fields.created
        issue_updated_date = None
        issue_resolved_date = issue.fields.resolutiondate

        # Check for SATD keywords in the issue's summary, description, and comments
        if issue_summary is None:
            issue_summary = ""
        if issue_description is None:
            issue_description = ""
        if issue_comments is None:
            issue_comments = ""
        if issue_created_date == None:
            issue_created_date = ""
        if issue_updated_date == None:
            issue_updated_date = ""
        if issue_resolved_date == None:
            issue_resolved_date = ""

        issue_type = issue.fields.issuetype.name
        issue_status = issue.fields.status.name
        issue_priority = issue.fields.priority.name

        # Create a JiraIssue instance for the SATD issue
        my_issue = JiraIssue(issue_key, issue_summary, issue_type, issue_status, issue_priority, issue_description, issue_comments, issue_created_date, issue_updated_date, issue_resolved_date)

        my_all_issues.add_issue(my_issue)

    return my_all_issues

def convert_date_jira_to_datetime(jira_date):
  regex = r"(\d{4})-(\d{2})-(\d{2})"
  match = re.match(regex, jira_date)

  if match:
      year, month, day = match.groups()
      datetime_object = datetime(year=int(year), month=int(month), day=int(day))
      return datetime_object
  else:
      return None

def convert_issues_to_dataframe(all_real_issues):
  l_issue_key_aux, l_issue_type_aux, l_issue_summary_aux, l_issue_description_aux, l_issue_status_aux, l_issue_priority_aux, l_issue_comments_aux = [], [], [], [], [], [], []
  l_issue_created_date, l_issue_resolved_date = [], []

  for issue in all_real_issues.get_issues():
    l_issue_key_aux.append(issue.key)
    l_issue_type_aux.append(issue.issue_type)
    l_issue_summary_aux.append(issue.summary)
    l_issue_description_aux.append(issue.description)
    l_issue_status_aux.append(issue.status)
    l_issue_priority_aux.append(issue.priority)
    texto_aux = ""
    for item in issue.get_comments():
      texto_aux = texto_aux + str(item) + "\n"
    l_issue_comments_aux.append(texto_aux)
    created_date_temp = convert_date_jira_to_datetime(issue.created_date)
    l_issue_created_date.append(created_date_temp)
    resolved_date_temp = convert_date_jira_to_datetime(issue.resolved_date)
    l_issue_resolved_date.append(resolved_date_temp)

  dict_all_reall_issues_in_commits_detailed = {
  'issue_key': l_issue_key_aux,
  'issue_type':l_issue_type_aux,
  'status':l_issue_status_aux,
  'priority':l_issue_priority_aux,
  'summary':l_issue_summary_aux,
  'description':l_issue_description_aux,
  'comments':l_issue_comments_aux,
  'created_date': l_issue_created_date,
  'resolved_date': l_issue_resolved_date
  }

  df_all_reall_issues_in_commits_detailed = pd.DataFrame(dict_all_reall_issues_in_commits_detailed)
  return df_all_reall_issues_in_commits_detailed

In [ ]:
def convert_commits_to_dataframe(dict_of_commits):
  '''
  v[0] = commit.msg,
  v[1] = data_commit,
  v[2] = commit.lines,
  v[3] = commit.files,
  v[4] = list_of_critical_files_modified,
  v[5] = list_of_modified_files,
  v[6] = list_dict_of_diff_files,
  v[7] = list_dict_of_diff_modified_files
  '''
  l_commit_hash, l_commit_msg, l_commit_data, l_commit_lines, l_commit_files, l_commit_critical_files, l_commit_modified_fies, l_commit_diff_files, l_commit_diff_modified_files = [], [], [], [], [], [], [], [], []
  for k, v in dict_of_commits.items():
    l_commit_hash.append(k)
    l_commit_msg.append(v[0])
    l_commit_data.append(v[1])
    l_commit_lines.append(v[2])
    l_commit_files.append(v[3])
    l_commit_critical_files.append(v[4])
    l_commit_modified_fies.append(v[5])
    l_commit_diff_files.append(v[6])
    l_commit_diff_modified_files.append(v[7])

  dict_of_commits_aux = {
      'hash': l_commit_hash,
      'msg': l_commit_msg,
      'date': l_commit_data,
      'lines': l_commit_lines,
      'files': l_commit_files,
      'critical_files': l_commit_critical_files,
      'modified_files': l_commit_modified_fies,
      'diff_files': l_commit_diff_files,
      'diff_files_modified_files': l_commit_diff_modified_files
  }

  df_commits = pd.DataFrame(dict_of_commits_aux)
  return df_commits

In [ ]:
def find_issues_id_by_project(input_string: str, project: str) -> list[str]:
    """Finds all Cassandra issue ID patterns in the input string.
    Args:
        input_string: The input string.
        project: The pattern related to project name, for example: CASSANDRA project name
    Returns:
        A list of project issue IDs, if found; otherwise, an empty list.
    """
    # Try to find all Cassandra issue ID patterns in the input string
    matches = re.findall(r"({0}-\d+)".format(project), input_string)
    # Return an empty list if no matches are found
    if not matches:
        return []
    # Convert the list of matches to a set to remove duplicates
    set_matches = set(matches)
    # Convert the set of matches back to a list
    list_unique_matches = list(set_matches)

    # Return the list of matched Cassandra issue IDs
    return list_unique_matches

def get_commits_with_critical_files_and_issues_in_this_commits(df_commits_with_critical_files):
  dict_issues_in_commits = {}
  for index in df_commits_with_critical_files.index:
    l_issues_in_commit = find_issues_id_by_project(input_string=df_commits_with_critical_files.msg[index], project=ISSUE_TRACKER_PROJECT)
    if len(l_issues_in_commit) > 0:
      commit_hash = df_commits_with_critical_files.hash[index]
      dict_issues_in_commits[commit_hash] = l_issues_in_commit

  list_issue_commits, list_issue_issues = [], []
  for k, v in dict_issues_in_commits.items():
    list_issue_commits.append(k)
    for issue in v:
      if issue not in list_issue_issues:
        list_issue_issues.append(issue)

  df_aux  = df_all_reall_issues_in_commits_detailed.copy()
  df_issues_in_commits_with_critical_classes = df_aux[df_aux['issue_key'].isin(list_issue_issues)]

  return dict_issues_in_commits, df_issues_in_commits_with_critical_classes

def calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size):
    # Calculate the Z-score for the given confidence level
    z_score = stats.norm.ppf(1 - (1 - confidence_level) / 2)

    # Calculate the sample size formula
    sample_size = ((z_score**2) * population_proportion * (1 - population_proportion)) / (margin_of_error**2)

    # Adjust for finite population
    if population_size:
        sample_size = sample_size / (1 + ((sample_size - 1) / population_size))

    return int(sample_size)

In [ ]:
def get_max_n_chars(text, max_n):
  text_length = len(text)
  if text_length <= max_n:
    return text
  else:
    return text[:max_n]

def create_new_file(filename, dir_name, issue_type, summary, description, status, comments):
  try:
    filename = dir_name + '/' + filename

    if issue_type is None:
      issue_type = ''
    if summary is None:
      summary = ''
    if description is None:
      description = ''
    if status is None:
      status = ''
    if comments is None:
      comments = ''

    with open(filename, mode='w') as f_issue:
      f_issue.write(f'issue_type: {issue_type} \n')
      f_issue.write(f'summary: {summary} \n')
      f_issue.write(f'description: {get_max_n_chars(text=description, max_n=1000)} \n')
      f_issue.write(f'status: {status} \n')
      f_issue.write(f'comments: {get_max_n_chars(text=comments, max_n=4000)} \n')
    print(f'File {filename} created with success!')

  except Exception as ex:
    print(f'Erro ao criar arquivo: {str(ex)}')


In [ ]:
# Seleciona randomicamente os issues para inspeção
def select_issues_to_inspection(sample_size, df_issues_in_commits_with_critical_classes, my_date='02/11/2023'):
  lista_issues_inspecao = []
  dict_issues_para_inspecao = {}
  list_issue_key = df_issues_in_commits_with_critical_classes.issue_key.to_list()
  list_issue_key = list(set(list_issue_key))
  sample_issues = random.choices(list_issue_key, k=sample_size)
  dict_issues_para_inspecao['02/11/2023'] = sample_issues
  print(f'{len(sample_issues)} para inspeção manual')

  date_file_name = my_date.split('/')
  date_file_name = date_file_name[0] + date_file_name[1] + date_file_name[2]
  file_name = 'issues_inspecao_' + date_file_name + '.txt'
  with open(file_name, mode='w') as f_temp:
    for v in dict_issues_para_inspecao[my_date]:
      elemento = v + ','
      f_temp.write(elemento)
  print(f'Relação de Issues salvos em {my_date} para inspeção.')
  return sample_issues

# Gera os arquivos .txt de cada issue selecionado para inspeção
def generate_files_issues_to_inspection(sample_issues, df_issues_in_commits_with_critical_classes):
  contador = 0
  my_dir_name = 'my_issues'
  if not os.path.exists(my_dir_name):
    os.makedirs(my_dir_name)

  total_of_issues = df_issues_in_commits_with_critical_classes.shape[0]
  for index in tqdm.tqdm(df_issues_in_commits_with_critical_classes.index, total=total_of_issues, desc='Analyzing issues'):
    for issue in sample_issues:
      if df_issues_in_commits_with_critical_classes.issue_key[index] == issue:
        create_new_file(filename=df_issues_in_commits_with_critical_classes.issue_key[index], dir_name=my_dir_name, issue_type=df_issues_in_commits_with_critical_classes.issue_type[index], summary=df_issues_in_commits_with_critical_classes.summary[index], description=df_issues_in_commits_with_critical_classes.description[index], status=df_issues_in_commits_with_critical_classes.status[index], comments=df_issues_in_commits_with_critical_classes.comments[index])
        contador += 1
  print(f'Foram criados {contador} arquivos para inspeção')

# 1. Extrai commits de um repositório de código

In [ ]:
start_date = datetime(2009, 3, 2, 0, 0, 0)
end_date = datetime(2023, 10, 4, 0, 0, 0)
extract_all_commits = get_all_commits_by_range(initial_date=start_date, final_date=end_date, repository_name=my_repository)

all_commits = extract_all_commits[0]
total_all_commits = extract_all_commits[1]

print(f'Total de commits extraídos: {total_all_commits}')

Wait...


Progress commit anlysis: 100%|██████████| 29230/29230 [13:10<00:00, 36.99it/s]

Total de commits extraídos: 29230


## Filtra os commits com classes críticas

In [ ]:
print(f'Analisa a faixa de commits entre: initial_date={str(start_date)}, final_date={str(end_date)}')
print('Registra apenas os commits que contem pelo menos um arquivo crítico')
filter_commits_with_critical_files = get_commits_by_range_and_critical_files(initial_date=start_date, final_date=end_date, critical_files=lista_arquivos_criticos, repository_name=my_repository)
commits_with_critical_files = filter_commits_with_critical_files[0]
total_of_commits_with_critical_files = filter_commits_with_critical_files[1]

print(f'Total de commits com classes críticas: {total_of_commits_with_critical_files}')

Analisa a faixa de commits entre: initial_date=2009-03-02 00:00:00, final_date=2023-10-04 00:00:00
Registra apenas os commits que contem pelo menos um arquivo crítico
Wait...


Progress critical files commit anlysis: 100%|██████████| 29230/29230 [06:02<00:00, 80.70it/s]

Total de commits com classes críticas: 29230


### Converte os commits com classes críticas para dataframe

In [ ]:
df_commits_with_critical_files = convert_commits_to_dataframe(dict_of_commits=commits_with_critical_files)
df_commits_with_critical_files

,hash,msg,date,lines,files,critical_files,modified_files,diff_files,diff_files_modified_files
0,1f91e99223b0d1b7ed8390400d4a06ac08e4aa85,git-svn-id: https://svn.apache.org/repos/asf/i...,2/3/2009,74502,344,"[DatabaseDescriptor.java, ColumnFamilyStore.ja...","[JSAP-2.1.jar, activation.jar, antlr-2.7.6.jar...","[{'DatabaseDescriptor.java': '@@ -0,0 +1,743 @...",[Binary files /dev/null and b/lib/JSAP-2.1.jar...
1,cf56f8f3acbeac0ddff9ee3131ad7abe2a919ba3,Add build script and make sure it works\n\ngit...,2/3/2009,156,4,[StorageService.java],"[build.xml, StorageService.java, DBTest.java, ...","[{'StorageService.java': '@@ -70,11 +70,6 @@ i...","[@@ -0,0 +1,133 @@\n+<?xml version=""1.0"" encod..."
2,5bea7e1772e9fd042b10abebb1e0c78579942b26,Removed references to JDK7 specific things.\n\...,20/3/2009,82,2,[ColumnFamilyStore.java],"[ColumnFamilyStore.java, Table.java]","[{'ColumnFamilyStore.java': '@@ -20,7 +20,6 @@...","[@@ -20,7 +20,6 @@ package org.apache.cassandr..."
3,f4f3ca50d84e165a909dca5608babb32df199f44,rename serializer2 -> serializerWithIndexers\n...,27/3/2009,26,4,[ColumnFamilyStore.java],"[ColumnFamily.java, ColumnFamilyStore.java, Me...","[{'ColumnFamilyStore.java': '@@ -22,10 +22,7 @...","[@@ -70,7 +70,7 @@ public final class ColumnFa..."
4,39d3f833f9c4e154eb157336f5e7b2a1ce9a79a3,add SuperColumn support to forceFlush. split ...,27/3/2009,89,5,[ColumnFamilyStore.java],"[ColumnFamilyStore.java, HintedHandOffManager....","[{'ColumnFamilyStore.java': '@@ -395,11 +395,1...","[@@ -395,11 +395,11 @@ public class ColumnFami..."
...,...,...,...,...,...,...,...,...,...
4517,6708499e4c930f7733894e6076b925fb151d9251,Remove metrics-reporter-config dependency\n\np...,12/9/2023,105,10,[CassandraDaemon.java],"[build-rat.xml, CHANGES.txt, NEWS.txt, jvm-ser...","[{'CassandraDaemon.java': '@@ -21,7 +21,6 @@ i...","[@@ -43,7 +43,6 @@\n <!-- Con..."
4518,20d80118ac02d853d501e3f5c24a63a12d4010f5,Add the ability to disable bulk loading of SST...,19/9/2023,289,11,[Config.java],"[CHANGES.txt, NEWS.txt, cassandra.yaml, Config...","[{'Config.java': '@@ -860,6 +860,7 @@ public c...","[@@ -1,4 +1,5 @@\n 5.1\n+ * Add the ability to..."
4519,d16e8d3653dce8ed767a040c06dbaabc47a9b474,Do not create sstable files before registering...,25/9/2023,724,18,[CompactionManager.java],"[CHANGES.txt, CompactionManager.java, Upgrader...","[{'CompactionManager.java': '@@ -1624,7 +1624,...","[@@ -1,4 +1,5 @@\n 5.0-alpha2\n+ * Do not crea..."
4520,b9586501a6b6cdfe465302448018785652c9b966,Internode legacy SSL storage port certificate ...,25/9/2023,237,13,[MessagingService.java],"[CHANGES.txt, InboundConnectionInitiator.java,...","[{'MessagingService.java': '@@ -213,6 +213,7 @...","[@@ -1,4 +1,5 @@\n 4.1.4\n+ * Internode legacy..."


In [ ]:
df_commits_with_critical_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4522 entries, 0 to 4521
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   hash                       4522 non-null   object
 1   msg                        4522 non-null   object
 2   date                       4522 non-null   object
 3   lines                      4522 non-null   int64 
 4   files                      4522 non-null   int64 
 5   critical_files             4522 non-null   object
 6   modified_files             4522 non-null   object
 7   diff_files                 4522 non-null   object
 8   diff_files_modified_files  4522 non-null   object
dtypes: int64(2), object(7)
memory usage: 318.1+ KB


# 2. Extrai issues de um issue tracker

## Configura ambiente para acessar Issue Tracker

In [ ]:
# Initialize the Jira connection
print('Initialize the Jira connection')
jira = JIRA(JIRA_SERVER, basic_auth=(username, password))

# Create a JiraUtils instance
print('Create a JiraUtils instance')
jira_utils = JiraUtils(ISSUE_TRACKER_PROJECT, jira)

date1 = (2009, 3, 2)
date2 = (2023, 10, 4)
distance = 120
print('Define date intervals')
print(f'From: {str(date1)} to: {date2}, by: {distance} days of distance.')

Initialize the Jira connection
Create a JiraUtils instance
Define date intervals
From: (2009, 3, 2) to: (2023, 10, 4), by: 120 days of distance.


## Recupera os issues do período dado

In [ ]:
# Fetch issues using date intervals
print('Fetch issues using date intervals')
block_of_issues = jira_utils.get_list_of_block_issues_by_dates(date1, date2, distance)

Fetch issues using date intervals
Aguarde...


Progress Message Analysis:   2%|▏         | 1/45 [00:03<02:34,  3.52s/it]

Range: project=CASSANDRA and created>="2009/03/02" and created<="2009/06/29", qtd issues: 263


Progress Message Analysis:   4%|▍         | 2/45 [00:06<02:25,  3.38s/it]

Range: project=CASSANDRA and created>="2009/06/30" and created<="2009/10/27", qtd issues: 251


...

Range: project=CASSANDRA and created>="2022/12/19" and created<="2023/04/17", qtd issues: 328


Progress Message Analysis:  98%|█████████▊| 44/45 [04:00<00:04,  4.00s/it]

Range: project=CASSANDRA and created>="2023/04/18" and created<="2023/08/15", qtd issues: 298


Progress Message Analysis: 100%|██████████| 45/45 [04:03<00:00,  5.41s/it]

Range: project=CASSANDRA and created>="2023/08/16" and created<="2023/12/13", qtd issues: 236
2023-11-04 15:27:35.450213
Tempo da consulta: 0:04:03.594157


In [ ]:
# Concatenate the block of issues into a single list
print('Concatenate the block of issues into a single list')
all_issues = jira_utils.concatenate_block_of_issues(block_of_issues)

Concatenate the block of issues into a single list
Total de issues recuperados: 18720


In [ ]:
all_real_issues = analyze_jira_all_issues(ISSUE_TRACKER_PROJECT, all_issues)

Progress jira all issues analysis: 100%|██████████| 18720/18720 [00:00<00:00, 77047.08it/s]


### Converte os issues recuperados para um dataframe

In [ ]:
df_all_reall_issues_in_commits_detailed = convert_issues_to_dataframe(all_real_issues)
df_all_reall_issues_in_commits_detailed

,issue_key,issue_type,status,priority,summary,description,comments,created_date,resolved_date
0,CASSANDRA-263,Bug,Resolved,Normal,get_slice needs to support desc from last column,At the moment there's no way to ask for a slic...,Is the problem that you don't know what the la...,2009-06-26,2009-07-31
1,CASSANDRA-262,Bug,Resolved,Normal,get_slice needs to allow returning all columns,Right now get_slice requires you to enter a 'l...,How about if we add a configuration parameter ...,2009-06-26,2009-07-04
2,CASSANDRA-261,Bug,Resolved,Normal,get_slice needs offset + limit,Right now get_slice does not allow you to prov...,"Me, explaining new get_slice: so now you would...",2009-06-26,2009-07-06
3,CASSANDRA-260,Bug,Resolved,Normal,Unable to read all columns in a column family ...,Since r788516 the typical 'get all columns for...,fixed in patch for CASSANDRA-277 (to be applie...,2009-06-26,2009-07-06
4,CASSANDRA-259,New Feature,Resolved,Normal,LRU cache for key positions,"add cache like the old touch cache, but workin...",LinkedHashMap is a nonstarter though. This wa...,2009-06-26,2009-07-06
...,...,...,...,...,...,...,...,...,...
18715,CASSANDRA-18769,Bug,Resolved,Normal,Fix flaky test: org.apache.cassandra.cql3.Memt...,{noformat}\r\njava.lang.reflect.InaccessibleOb...,I think this is a duplicate of CASSANDRA-18447...,2023-08-16,2023-08-16
18716,CASSANDRA-18768,New Feature,Triage Needed,Normal,Harry changes required for CEP-21 testing (NTS...,,,2023-08-16,NaT
18717,CASSANDRA-18767,Bug,Resolved,Normal,tablestats should show speculative retries,NodeProbe makes [provisions|https://github.com...,||Branch||CI||\r\n|[3.0|https://github.com/dri...,2023-08-16,2023-08-18
18718,CASSANDRA-18766,Bug,Open,Normal,high speculative retries on v4.1.3,There are up to 10+ times higher speculative r...,These are changes between 4.1.2 and 4.1.3 \r\n...,2023-08-16,NaT


In [ ]:
print('Salva os dados de df_all_reall_issues_in_commits_detailed no arquivo cassandra_all_issues_in_commits.xlsx')

df_all_reall_issues_in_commits_detailed['time_resolution'] = df_all_reall_issues_in_commits_detailed['resolved_date'] - df_all_reall_issues_in_commits_detailed['created_date']

In [ ]:
print('Gera arquivo .csv cassandra_all_issues_in_commits')
colunas = ['issue_key', 'issue_type', 'status', 'summary','created_date', 'resolved_date', 'time_resolution']
df_all_reall_issues_in_commits_detailed[colunas].to_csv('cassandra_all_issues_in_commits.csv', index=False)

### Recupera Issues com referência nos commits com classes críticas

In [ ]:
dict_issues_in_commits, df_issues_in_commits_with_critical_classes = get_commits_with_critical_files_and_issues_in_this_commits(df_commits_with_critical_files)
df_issues_in_commits_with_critical_classes

,issue_key,issue_type,status,priority,summary,description,comments,created_date,resolved_date
2,CASSANDRA-261,Bug,Resolved,Normal,get_slice needs offset + limit,Right now get_slice does not allow you to prov...,"Me, explaining new get_slice: so now you would...",2009-06-26,2009-07-06
4,CASSANDRA-259,New Feature,Resolved,Normal,LRU cache for key positions,"add cache like the old touch cache, but workin...",LinkedHashMap is a nonstarter though. This wa...,2009-06-26,2009-07-06
9,CASSANDRA-254,Improvement,Resolved,Normal,clean up sstable constructors,,+1\ncommitted\nIntegrated in Cassandra #120 (S...,2009-06-24,2009-06-25
12,CASSANDRA-251,Improvement,Resolved,Normal,Restore ColumnSort=Time on Super Columnfamilies,Setting ColumnSort=Time on a Super Columnfamil...,"Sorry, I got a little rattled by the ""oh crap ...",2009-06-24,2009-07-06
15,CASSANDRA-248,Improvement,Resolved,Normal,Replace factory method based on explicit locks...,Replace factory method based on explicit locks...,Patch submitted.\nThis is double-checked locki...,2009-06-21,2009-11-20
...,...,...,...,...,...,...,...,...,...
18434,CASSANDRA-18504,Improvement,Resolved,Normal,"Added support for type VECTOR<type, dimension>","Based off several mailing list threads (see ""[...",having a hard time fixing CQL3TypeLiteralTest ...,2023-05-05,2023-06-22
18438,CASSANDRA-18500,New Feature,Resolved,Normal,Add guardrail for partition size,"Add a guardrail for max partition size, for ex...",The proposed patch adds a new guardrail for pa...,2023-05-04,2023-05-25
18668,CASSANDRA-18816,Improvement,Resolved,Normal,Add support for repair coordinator to retry me...,"Now that CASSANDRA-15399 is in, most of the re...",scope slightly increased due to bugs found by ...,2023-09-01,2023-09-27
18681,CASSANDRA-18803,Bug,Resolved,Normal,Refactor validation logic in StorageService.re...,This is a follow-up ticket of CASSANDRA-14319,[~aweisberg] would you mind to take a look? Su...,2023-08-29,2023-08-31


In [ ]:
df_issues_in_commits_with_critical_classes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2912 entries, 2 to 18703
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   issue_key      2912 non-null   object        
 1   issue_type     2912 non-null   object        
 2   status         2912 non-null   object        
 3   priority       2912 non-null   object        
 4   summary        2912 non-null   object        
 5   description    2912 non-null   object        
 6   comments       2912 non-null   object        
 7   created_date   2912 non-null   datetime64[ns]
 8   resolved_date  2904 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(7)
memory usage: 227.5+ KB


In [ ]:
df_issues_in_commits_with_critical_classes['time_resolution'] = df_issues_in_commits_with_critical_classes['resolved_date'] - df_issues_in_commits_with_critical_classes['created_date']

In [ ]:
print('Gera arquivo .csv cassandra_issues_in_commits_arquivos_criticos')
df_issues_in_commits_with_critical_classes[colunas].to_csv('cassandra_issues_in_commits_arquivos_criticos.csv', index=False)

# Seleciona amostra dos Issues

In [ ]:
confidence_level = 0.95
margin_of_error = 0.05
population_proportion = 0.8
population_size = len(df_issues_in_commits_with_critical_classes)

sample_size = calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size)
print(f'População de issues: {population_size}')
print(f"Sample size para inspeção: {sample_size}")

População de issues: 2912
Sample size para inspeção: 226


In [ ]:
sample_issues = select_issues_to_inspection(sample_size, df_issues_in_commits_with_critical_classes, my_date='02/11/2023')

226 para inspeção manual
Relação de Issues salvos em 02/11/2023 para inspeção.


In [ ]:
generate_files_issues_to_inspection(sample_issues, df_issues_in_commits_with_critical_classes)

Analyzing issues:   2%|▏         | 49/2912 [00:00<00:05, 483.22it/s]

File my_issues/CASSANDRA-241 created with success!
File my_issues/CASSANDRA-184 created with success!
File my_issues/CASSANDRA-181 created with success!
File my_issues/CASSANDRA-154 created with success!
File my_issues/CASSANDRA-111 created with success!
File my_issues/CASSANDRA-96 created with success!
File my_issues/CASSANDRA-37 created with success!
File my_issues/CASSANDRA-463 created with success!


Analyzing issues:   3%|▎         | 98/2912 [00:00<00:06, 466.59it/s]

File my_issues/CASSANDRA-444 created with success!
File my_issues/CASSANDRA-435 created with success!


Analyzing issues:   5%|▌         | 147/2912 [00:00<00:05, 475.35it/s]

...

File my_issues/CASSANDRA-17256 created with success!
File my_issues/CASSANDRA-17790 created with success!
File my_issues/CASSANDRA-17680 created with success!
File my_issues/CASSANDRA-17592 created with success!
File my_issues/CASSANDRA-18102 created with success!
File my_issues/CASSANDRA-18055 created with success!


Analyzing issues: 100%|█████████▉| 2898/2912 [00:07<00:00, 265.07it/s]

File my_issues/CASSANDRA-18023 created with success!
File my_issues/CASSANDRA-17868 created with success!
File my_issues/CASSANDRA-17851 created with success!
File my_issues/CASSANDRA-18441 created with success!
File my_issues/CASSANDRA-18430 created with success!
File my_issues/CASSANDRA-18219 created with success!
File my_issues/CASSANDRA-18153 created with success!
File my_issues/CASSANDRA-18626 created with success!


Analyzing issues: 100%|██████████| 2912/2912 [00:07<00:00, 396.80it/s]

File my_issues/CASSANDRA-18550 created with success!
Foram criados 226 arquivos para inspeção


In [ ]:
!zip -r my_issues.zip my_issues

updating: my_issues/ (stored 0%)
  adding: my_issues/CASSANDRA-5125 (deflated 60%)
  adding: my_issues/CASSANDRA-13013 (deflated 54%)
  adding: my_issues/CASSANDRA-111 (deflated 54%)
  adding: my_issues/CASSANDRA-1788 (deflated 54%)
  adding: my_issues/CASSANDRA-5556 (deflated 40%)
  adding: my_issues/CASSANDRA-6948 (deflated 58%)
  adding: my_issues/CASSANDRA-9554 (deflated 53%)
  adding: my_issues/CASSANDRA-18102 (deflated 58%)
  adding: my_issues/CASSANDRA-2411 (deflated 45%)
  adding: my_issues/CASSANDRA-13653 (deflated 52%)
  adding: my_issues/CASSANDRA-10023 (deflated 51%)
  adding: my_issues/CASSANDRA-1721 (deflated 37%)
  adding: my_issues/CASSANDRA-9631 (deflated 58%)
  adding: my_issues/CASSANDRA-13216 (deflated 63%)
  adding: my_issues/CASSANDRA-17680 (deflated 59%)
  adding: my_issues/CASSANDRA-2468 (deflated 55%)
  adding: my_issues/CASSANDRA-5148 (deflated 41%)
  adding: my_issues/CASSANDRA-6809 (deflated 55%)
  adding: my_issues/CASSANDRA-181 (deflated 45%)
  adding: my_